In [ ]:
import numpy as np
from numba import jit

import matplotlib.pyplot as plt

## Introduction to miniMD

### Implementing your own systems

Important for any molecular dynamics or monte carlo simulation are the potential energy function and force function of the system and the choice of the intrgrator. For the latter, we here use simple overdamped langevin dynamics with the intend of making the implementation easy to read. The integrator is already implemented, nevertheless, we will dicuss it in a second. All implementations are heavily based on numpy and we use Numba to just-in-time compile the functions for an increase in performance. If you run into error messages, it can be very useful to disable the jit-compilation by commenting the '@jit(nopython=True)' line.

The force and energy of the system is the part that you will have to modify frequently. The functions below show an example implementation for an harmonic oscillator:



In [ ]:

@jit(nopython=True) 
def custom_potential_energy(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy given a configuration current_x. 
    The example here is for a harmonic oscillator with a force constant of 1.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    U : float
        Potential energy of the configuration

    """

    potential_energy = 0.5 * current_x**2

    return potential_energy


@jit(nopython=True) 
def custom_force_function(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force given a configuration current_x. The example here 
    is for a harmonic oscillator with a force constant of 1.

    Remember: The force is the negative gradient of the potential energy with 
    respect to the current configuration. Therefore, it must be of the same 
    dimensionality as the configuration.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    force : np.ndarray
        Force corresponding the provided configuration.

    """
    
    force = -current_x

    return force


You can see that the potential energy function takes as an argument a numpy array and maps it to a float. Compared to that, the force, defined as the negative gradient of the energy with respect to the positions, returns also a numpy array.

### The simulation code

For the simulation, we use a simple overdamped langevin integrator that updates the particles position based on the force $f(x)$, the diffusion coefficient $D$ and the timestep $\Delta t$:
$$x_{i+1} = x_i + \beta D f(x) \Delta t + \sqrt{2 D \Delta t} * g$$
where g is a random number drawn from the standard normal distribution.

The routine to update the positions will usually be hidden in 'miniMD.py'. However, we can have a look at it here:

In [ ]:

import numpy as np
from numba import jit

@jit(nopython=True) 
def update_positions(current_x : np.ndarray, force : np.ndarray, beta : float, dt : float, diffusion_coeff : float) -> np.ndarray:
    """
    Update the positions using overdamped langevin dynamics:

        x_(i+1) = x_i + D * beta * force * dt + sqrt(2 * D * dt) * g

    where x_i are the positions at i (current_x) and x_(i+1) are the updated positions (next_x). 
    The diffusion coefficient D (diffusion_coeff), the timestep dt, the temperature in form of 
    beta and the force are needed for the propagation. The factor g is a random number from a 
    standard normal distribution.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.
        
    force : np.ndarray
        The force corresponding to current_x. This has to be of the same shape as current_x. 

    beta : float
        Beta determines the simulation temperature, it is equivalent to 1/kT. Must be greater than 0. 

    dt : float
        The simulation timestep for the propagation of current_x. Must be greater than 0. Decrease this or diffusion_coeff if you experience an unstable configuration.
    
    diffusion_coeff : float
        The diffusion coefficient for the propagation of current_x determining the magnitude of random "bumps". Must be greater than 0. Decrease this or diffusion_coeff if you experience an unstable configuration.
        
    Returns
    -------
    new_x : np.ndarray
        Updated Configuration.

    """

    assert dt > 0, "Timestep must be positive."
    assert diffusion_coeff > 0, "Diffusion coefficient must be positive."
    assert beta > 0, "Temperature must be positive."
    assert current_x.shape == force.shape, "Force and position vector must be of the same size, check your force function."

    # Draw random number from a standard normal distribution
    gauss_rand = np.random.randn(*current_x.shape).astype(np.float32)
    
    # Calculate displacement
    prefactor = np.sqrt(2 * diffusion_coeff * dt)
    dx = diffusion_coeff * dt * force * beta + prefactor * gauss_rand

    # Update the configuration x
    next_x = current_x + dx
    
    return next_x

The function takes the current positions in form of current_x and the force, ideally calculated using your implementation, and updates the positions. Again, if you later on see strange errors, try to comment the jit-line. Now for a first exercise:

**1) Given the x_values and the plotting script below, make a plot of the potential energy and the force as a function of x_values.**

In [ ]:
x_values = np.linspace(-2, 2, 100)

energies = ...
forces = ...

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(5,2.5), dpi=180)

ax[0].plot(energies, lw=1)
ax[1].plot(forces, lw=1)

ax[0].set_xlabel("x")
ax[0].set_ylabel("U(x)")

ax[1].set_xlabel("x")
ax[1].set_ylabel("f(x)")

ax[0].set_title("Potential Energy")
ax[1].set_title("Force")
plt.tight_layout()

### Running a simulation

Next up we want to code an actual simulation. Below, we have prepared for you some settings for an MD simulation, the comments describe their desired role:


In [ ]:
# Number of integration steps in total
total_steps = 5000000

# Number of steps for equilibration before we record the trajectory, sometimes called 'burn-in time'
equilibration_steps = 10

# How many integration steps are between two frames
output_frequency = 1

beta = 1 # beta = 1/(k_B*T)
timestep = 0.001
diffusion_coefficient = 1

# The initial positions to start the simulation, here just a 1D array of zeros
initial_x = np.zeros(1)

# Some sanity checks :)
assert equilibration_steps < total_steps, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < total_steps, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


**2) With these variables in mind, write a small MD code that outputs a trajectory.** It should produce a trajectory, that means a list or (better) an array of frames corresponding to each time slice.

In [ ]:
# Initialize variables

...

# Run Simulation
for step in range(total_steps):

    ...

**3) Plot the trajectory as a function of the frame number.**

In [ ]:
trajectory = ...

# Plotting
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

ax.plot(trajectory, lw=1)

ax.set_xlabel("Frame")
ax.set_ylabel("x")

plt.show()

**4) Plot the energy of each configuration on the trajectory as a function of the frame number.**

In [ ]:

energy_trajectory = ...


# Plotting
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

ax.plot(energy_trajectory, lw=1)

ax.set_xlabel("Frame")
ax.set_ylabel("U(x)")

plt.show()

### Analysing the stationary distribution

The simulation should produce configurations distributed according to the Boltzmann distribution:

$$p(x) = Z^{-1} e^{-\beta U(x)}$$

where Z is the partition function normalizing the density function:

$$Z = \int_{-\infty}^{\infty} \text{d}x\ e^{-\beta U(x)}$$

For our harmonic oscillator at $\beta$=1, the probability density function might remind you of something:

$$p(x) = Z^{-1} e^{-\frac{1}{2} x^2}$$

**5) Extract an estimate of the probability density along $x$ from the trajectory using 'np.histogram'. Compare the density to the reference density by calculating $Z$ and using the provided plotting code.**

In [ ]:
bins = np.linspace(-2, 2, 50)
bin_centers = ...

density, bin_edges = np.histogram(...)

reference_density = ...

# Plotting
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

ax.plot(bin_centers, reference_density, lw=1, c="0", ls="dashed")
ax.plot(bin_centers, density, lw=1)

ax.set_xlabel("x")
ax.set_ylabel(r"p(x)")

plt.show()

### The simplest way towards a free energy

The we can use the relation between the free energy and the probability density to obtain an estimate of the free energy along x:

$$A(x) = - \frac{1}{\beta} \ln [p(x)] $$

This is often in literature referred to as 'Boltzmann inversion'.

**6) Estimate the free energy as a function of $x$ based on the previously obtained density. Calculate the reference free energy and compare it to the estimated free energy  using the provided plotting code.**

In [ ]:
bins = np.linspace(-2, 2, 50)
bin_centers = ...

density, bin_edges = ...

free_energy = ...
reference_free_energy = ...

fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

ax.plot(bin_centers, reference_free_energy, lw=1, c="0", ls="dashed")
ax.plot(bin_centers, free_energy, lw=1)

ax.set_xlabel("x")
ax.set_ylabel(r"A(x)")

plt.show()

### Optional:

1. Change the potential energy function to something more interesing than a harmonics oscillator. Suggestions could be:
$$a * x^4 - x^2$$
$$\sin(a*x) + x^2$$
where a is a constant parameter. What do you see in these systems? Try you own potential forms!

2. In the following tutorials we use 2D systems, try to extend your code with minimal additions for e.g. a 2D harmonic oscillator.